<a href="https://colab.research.google.com/github/SepideSohrabi/K-fold-Image-Classification/blob/main/2th_customized_k_fold_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
#2

# Make it False if do not intend to use Google Colab and want to train in local machine!!
google_colab_flag = True

# For training in Google Colab
if(google_colab_flag):
    from google.colab import drive
    drive.mount('/content/drive')
    !ls
    import sys
    # This is the path to where in google drive the code is stored!
    
    root_path = '/content/drive/MyDrive/AmnAzmoonPics/'
    sys.path.append(root_path)

# For local training
else:
    root_path = ''



Mounted at /content/drive
drive  sample_data


In [12]:
#3
from PIL import Image
import glob
image_legal = []
for filename in glob.glob('/content/drive/MyDrive/AmnAzmoonPics/DS/train/Legal/*.png'): #assuming gif
    im=Image.open(filename)
    image_legal.append(im)

In [13]:
#4
image_ilegal = []
for filename in glob.glob('/content/drive/MyDrive/AmnAzmoonPics/DS/train/iLegal/*.png'): #assuming gif
    im=Image.open(filename)
    image_ilegal.append(im)

In [ ]:
#5
#for preprocessing 
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])


In [ ]:
#6
X_iLegal=[]
for img in image_ilegal:
    transformed_img=transform(img)
    X_iLegal.append(transformed_img)


In [ ]:
#7
X_Legal=[]
for img in image_legal:
  transformed_img=transform(img)
  X_Legal.append(transformed_img)

In [ ]:
#8
import torch 
# Load the ResNext-WSL model

# small size model
#model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')
# medium size
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x16d_wsl')
# large size
# model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x32d_wsl')
# very large
# model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x48d_wsl')
# model.eval()

In [ ]:
#9
model.eval()
out_Legal=[]
for i in range (len(X_Legal)):
    batch_t = torch.unsqueeze(X_Legal[i], 0)
    out_Legal.append(model(batch_t).detach().numpy())


In [ ]:
#10
out_iLegal=[]
for i in range (len(X_iLegal)):
    batch_t = torch.unsqueeze(X_iLegal[i], 0)
    out_iLegal.append(model(batch_t).detach().numpy())


In [ ]:
#11
out_Legal_array=np.asarray(out_Legal)
out_iLegal_array=np.asarray(out_iLegal)

In [ ]:
#12
X_train=np.concatenate((out_Legal_array,out_iLegal_array) , axis=0)
X_train=np.asarray(X_train)
X_train=np.squeeze(X_train)

In [ ]:
#13
#1 refers to legal class and 0 refers to ilegal class
y_train=[]
for i in range (45):
  legal=1
  y_train.append(legal)


for i in range (46 ):
  ilegal=0
  y_train.append(ilegal)

y_train=np.asarray(y_train)

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train , y_train)